In [38]:
!pip install scikit-learn sklearn_crfsuite

In [39]:
def transform_dataset(page_dataset, for_inference):
    labeled_text_dataset = []
    for page in page_dataset:
        task_id = page["taskId"]
        page_words = page["representativeData"]["page_data_words"]
        
        geo_dictionary = {}
        if not for_inference:
            page_answers = page.get("answers")
            for page_answer in page_answers[0]["answer"]:
                geo_label = page_answer["id"]
                for geo_part in page_answer["data"]:
                    for index in range(geo_part["start"], geo_part["end"]):
                        geo_dictionary[index] = geo_label
        
        labeled_text = []
        for word_index, word in enumerate(page_words):
            word_label = "0" if for_inference else geo_dictionary.get(word_index, "O")
            labeled_text.append((word, word_label))
            
        labeled_text_dataset.append((task_id, labeled_text))
    
    return labeled_text_dataset

In [40]:
import json

def get_labeled_dataset(dataset_path, for_inference=False):
    with open(dataset_path) as dataset:
        train_dataset = json.load(dataset)
        
    labeled_dataset = transform_dataset(train_dataset["data"]["results"], for_inference)
    return labeled_dataset

In [41]:
def word2features(text, i):
    word: str = text[i][0]
    
    features = {
        "word.lower()": word.lower(),
        "word[-2:]": word[-2:],
        "word[-3:]": word[-3:],
        "word.isupper()": word.isupper(),
        "word.istitle()": word.istitle(),
        "word.isdigit()": word.isdigit()
    }
    
    if i > 0:
        another_word = text[i - 1][0]
        features.update({
            "-1:word.lower()": another_word.lower(),
            "-1:word.istitle()": another_word.istitle(),
            "-1:word.isupper()": word.isupper()
        })
    else:
        features["BOS"] = True
        
    if i < len(text) - 1:
        another_word = text[i + 1][0]
        features.update({
            "+1:word.lower()": another_word.lower(),
            "+1:word.istitle()": another_word.istitle(),
            "+1:word.isupper()": another_word.isupper()
        })
    else:
        features["EOS"] = True
        
    return features

In [42]:
def text2features(text):
    return [word2features(text, index) for index in range(len(text))]

def text2labels(text):
    return [label for _, label in text]

In [43]:
def get_validation_result(X_validation, y_pred):
    validation_result = []
    
    for ((task_id, labeled_text), predictions) in zip(X_validation, y_pred):
        answers = {}
        current_label = None
        start_index = None
        
        for current_index, label in enumerate(predictions):
            if label == current_label:
                continue
            else:
                if current_label is not None and current_label != "O":
                    if current_label not in answers:
                        answers[current_label] = []
                    answers[current_label].append({"start": start_index, "end": current_index})
                
                if label != "0":
                    current_label = label
                    start_index = current_index
                else:
                    current_label = None
    
        if current_label is not None and current_label != "O":
            if current_label not in answers:
                answers[current_label] = []
            answers[current_label].append({"start": start_index, "end": len(predictions)})
        
        validation_answers = []
        for label, segments in answers.items():
            validation_answers.append({"id": label, "data": segments})
        
        validation_result.append({
            "taskId": task_id,
            "answer": validation_answers
        })
        
    return validation_result

In [44]:
train_dataset = get_labeled_dataset("datasets/train_geo_extractor.json")

X_train = [text2features(text) for _, text in train_dataset]
y_train = [text2labels(text) for _, text in train_dataset]

In [45]:
import sklearn_crfsuite

crf_model = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

try:
    crf_model.fit(X_train, y_train)
except AttributeError:
    pass

In [46]:
test_dataset = get_labeled_dataset("datasets/test_geo_extractor.json")

X_test = [text2features(text) for _, text in test_dataset]
y_test = [text2labels(text) for _, text in test_dataset]

In [47]:
from sklearn.metrics import classification_report, matthews_corrcoef

y_pred = crf_model.predict(X_test)
y_test_flat = [label for text in y_test for label in text]
y_pred_flat = [label for text in y_pred for label in text]

report = classification_report(y_test_flat, y_pred_flat)
matthews_correlation_coefficient = matthews_corrcoef(y_test_flat, y_pred_flat)

print(report)
print(f"Matthews Correlation Coefficient: {matthews_correlation_coefficient}")

                   precision    recall  f1-score   support

                O       0.99      0.99      0.99     62822
     central_city       0.41      0.17      0.24       184
      geo_address       0.84      0.71      0.77      1040
     geo_building       0.82      0.69      0.75       453
         geo_city       0.81      0.83      0.82      1433
     geo_district       0.85      0.74      0.80       387
geo_microdistrict       0.65      0.48      0.55       382
       geo_region       0.99      0.98      0.99      1733
geo_region_oblast       0.84      0.84      0.84       297
       geo_street       0.81      0.73      0.77      1059

         accuracy                           0.97     69790
        macro avg       0.80      0.72      0.75     69790
     weighted avg       0.97      0.97      0.97     69790

Matthews Correlation Coefficient: 0.8557235324761806


In [48]:
validation_dataset = get_labeled_dataset("datasets/val_no_answer_geo_extractor.json", for_inference=True)

X_validation = [(task_id, text2features(text)) for task_id, text in validation_dataset]

X_validation_features = [text_features for _, text_features in X_validation]

y_pred = crf_model.predict(X_validation_features)

In [51]:
validation_result = get_validation_result(X_validation, y_pred)

with open("validation_result.json", "w", encoding="utf-8") as file:
    json.dump(validation_result, file, ensure_ascii=False, indent=4)

print("Validation result has been saved!")

Validation result has been saved!
